- request模块
    - 概念：一个基于网络请求的模块，用来模拟浏览器发起请求
    - 编码流程：
        - 指定url
        - 进行请求的发送
        - 获取响应数据（爬取到的数据）
        - 持久化存储
            - 环境安装：pip install requests

- 爬取搜狗首页对应的页面源码数据

In [10]:
import requests

# https://www.sogou.com/
url = r"https://www.sogou.com/"
# 发起请求，返回值是一个Response对象
res = requests.get(url)
# 返回字符串形式的数据
page_text = res.text
# 存储数据
with open(r"./Sougou.html","w",encoding="utf8") as f:
    f.write(page_text)

- 基于搜狗编写一个简易的网页采集器

In [13]:
# 将url携带的参数设定成动态变化
url = r"https://www.sogou.com/web"
# 存储动态的请求参数
wd = input("Enter a keyword")
params = {
    "query": wd
}
# 将params作用到请求中
# params参数：对请求url参数的封装
res = requests.get(url,params=params)
res.encoding = "utf-8"
page_text = res.text
with open("./searchResult.html","w",encoding="utf8") as f:
    f.write(page_text) 

Enter a keyword周杰伦


- 反爬机制：User-Agent
- 反反爬机制：UA伪装

In [15]:
# 将url携带的参数设定成动态变化
url = r"https://www.sogou.com/web"
# 存储动态的请求参数
wd = input("Enter a keyword")
params = {
    "query": wd
}
# 请求头信息
headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"
}
# 将params作用到请求中
# params参数：对请求url参数的封装
# headers 参数：实现UA伪装
res = requests.get(url,params=params,headers=headers)
res.encoding = "utf-8"
page_text = res.text
with open("./searchResult.html","w",encoding="utf8") as f:
    f.write(page_text) 
print("爬取完成！")

Enter a keyword周杰伦
爬取完成！


- 爬取豆瓣电影详情数据
    - 分析：当滚轮滑动到底部时，发起了一个Ajax请求，且该请求到了一组电影数据
- 动态加载数据：通过另一个额外的请求去请求到的数据
    - Ajax请求生产动态加载数据
    - js生成动态加载数据

In [22]:
# 链接参数：type=5&interval_id=100%3A90&action=&start=0&limit=20
url = r"https://movie.douban.com/j/chart/top_list"

# 输入
start = input("请输入开始编号：")
limit = input("请输入搜索范围：")

# 参数
params = {
    "type":5,
    "interval_id":"100:90",
    "action":"",
    "start":start,
    "limit":limit
}
# 定义headers，模拟User-Agent
headers = {
    "User-Agent":r"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1"
}
response = requests.get(url,params=params,headers=headers)
response.encoding = "utf8"

# json返回的是序列化好的对象
fp = open("./movie.txt","w+",encoding="utf8")
data_list = response.json()
for dic in data_list:
    title = dic["title"]
    score = dic["score"]
    fp.write(title+":"+score+"\n")
fp.close()
print("爬取成功")


请输入开始编号：10
请输入搜索范围：30
爬取成功


- 爬取肯德基餐厅位置信息

In [37]:
import re

# http://www.kfc.com.cn/kfccda/ashx/GetStoreList.ashx?op=keyword
# keyword: 城市
city = input("请输入城市：")
post_url = r"http://www.kfc.com.cn/kfccda/ashx/GetStoreList.ashx?op=keyword"
data = {
    "cname":"",
    "pid":"",
    "keyword":city,
    "pageIndex":"1",
    "pageSize":"10"
}
# 定义headers，模拟User-Agent
headers = {
    "User-Agent":r"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1"
}
# data参数：表示get()方法中的params参数
response = requests.post(post_url,data,headers=headers)
kfc_data = response.text
# print(kfc_data)
# 使用re模块获取数据
dataList = re.findall(r'"addressDetail":".*?"',kfc_data)
print(dataList)


请输入城市：北京
['"addressDetail":"小营东路3号北京凯基伦购物中心一层西侧"', '"addressDetail":"朝阳路杨闸环岛西北京通苑30号楼一层南侧"', '"addressDetail":"黄寺大街15号北京城乡黄寺商厦"', '"addressDetail":"西四环北路117号北京欧尚超市F1、B1"', '"addressDetail":"北京经济开发区西环北路18号F1＋F2"', '"addressDetail":"通顺路石园西区南侧北京顺义西单商场石园分店一层、二层部分"', '"addressDetail":"北京南站候车大厅B岛201号"', '"addressDetail":"北京北清路1号146区"', '"addressDetail":"海户屯北京新世纪服装商贸城一层南侧"', '"addressDetail":"巴沟路2号北京华联万柳购物中心一层"']


### 如何判定一张页面中是否存在动态加载数据?
- 抓包工具进行局部搜索
- 若判定出页面中有动态加载的数据,如何进行数据定位?
    - 使用抓包工具进行全局搜索
- 对一个陌生的网站数据进行爬取前,要判断爬取的数据是否为动态加载

- 爬取药监总局化妆品许可证数据(http://scxk.nmpa.gov.cn:81/xk/)
    - 分析:
        - 1. 网站首页和企业的详情页数据都是动态加载
        - 2. 分析某一家企业详情数据是怎么来的
            - 企业详情数据是通过一个Ajax请求(post)请求到的
            - 请求对应url:http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsById
            - 该请求携带了参数:id:d80612543cd9413b84e7525d771cee5f
            - 结论:
                - 1.每家企业都是post形式的Ajax请求到的
                - 2.每家企业对应的Ajax请求的url都一样,请求方式都是post,只有请求参数id的值不一样
                - 3.需要获取每一家企业的id值,即可获取每一家企业对应的详情数据
        - 3. 需要获取每家企业的id值
            - 思路;每一家企业的id值应该在首页对应相关请求或响应中
                 首页请求url:http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsList
                 参数:on: true
                    page: 1
                    pageSize: 15
                    productName: 
                    conditionType: 1
                    applyname: 
                    applysn: 
            - 结论:每一家企业的id值,存储在首页中的某一个Ajax请求对应的响应中,需要将该响应数据中的id值提取出来即可

In [44]:
# 每一家企业的id
url = r"http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsList"
data = {
    "on": "true",
    "page": "1",
    "pageSize": "15",
    "productName": "",
    "conditionType": "1",
    "applyname": "",
    "applysn": ""
}
headers = {
    "User-Agent":r"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1"
}
# 每一家企业的id
data_json = requests.post(url,data,headers=headers).json()
data_list = data_json["list"]
fp = open("./companydata.txt","w+",encoding="utf8")
for l in data_list:
    com_id = l["ID"]
    # print(com_id)
    post_url = r"http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsById"
    post_data = {
        "id":com_id
    }
    res = requests.post(post_url,post_data,headers=headers)
    details = res.json()
    companyName = details["epsName"]
    companyAddress = details["epsAddress"]
    fp.write(companyName+":"+companyAddress+"\n")
print("爬取完成")
fp.close()

- 什么是抓包工具
    - 代理服务器:进行请求转发

- 作业:将药监总局前5页企业详情数据进行爬取,且进行任意形式的持久化存储

In [47]:
indexUrl = r"http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsList"
detailUrl = r"http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsById"
headers = {
    "User-Agent":r"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1"
}
f = open("./practiceDrug.txt","w+",encoding="utf8")
for i in range(1,6):
    indexData = {
        "on":"true",
        "page":i,
        "pageSize":"15",
        "productName":"",
        "conditionType":"1",
        "applyname":"",
        "applysn":""
    }
    dataList = requests.post(indexUrl,indexData,headers=headers).json()["list"]
    for ls in dataList:
        companyId = ls["ID"]
        detailData = {
            "id":companyId
        }
        details = requests.post(detailUrl,detailData,headers=headers).json()
        companyName = details["epsName"]
        companyAddress = details["epsProductAddress"]
        f.write(companyName+","+companyAddress+"\n")
f.close()
print("爬取完成")

爬取完成
